In [1]:
from gurobipy import Model, GRB, quicksum
import pandas as pd
import numpy as np

def read_basic_data():
    # Excel file names
    travel_times = pd.read_excel('a2_part1.xlsx','Travel Times')
    lines = pd.read_excel('a2_part1.xlsx', 'Lines')

    lines = lines.drop(['Name','Frequency'], axis = 1)
    lines = lines.fillna(0)

    weights = 1 #Each activity has the same importance for us
    
    return travel_times, lines, weights



In [2]:
def build_model(travel_times, lines, weights):
    T = 30

    # Initialize the Gurobi model
    model = Model("NS_trains")

    line_names = ['800','3000','3100','3500','3900']
    running_activites_f = {}
    running_activites_b = {}
    dwelling_activities_f = {}
    dwelling_activities_b = {}
    transfer_activities = {}
    sync_activities = {}
    headway_activities = {}
    events_pi_f= {}
    events_pi_b= {}

    for k in range(5):
        station_names = lines.iloc[k].tolist()  # Extract station names from the first row
        number = line_names[k]

        # Create Gurobi variables for each possible trip between stations
        for i in range(len(station_names) - 1):
            station1 = str(station_names[i])
            station2 = str(station_names[i+1])
            station3 = str(station_names[i-1])

            if station1 == '0' or station2 == '0':
                break
                
            # Running activities forward (ra)
            running_activites_f[f'ra-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                        name=f'ra-{station1}-{station2}-{number}')
            # Running activities backwards (ra)
            running_activites_b[f'ra-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                        name=f'ra-{station2}-{station1}-{number}')
            

            if i != 0 and i != len(station_names):
                # Dwelling activities forward (daf)
                dwelling_activities_f[f'daf-{station3}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'daf-{station3}-{station1}-{number}')
                # Dwelling activities backwards (dab)
                dwelling_activities_b[f'dab-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'dab-{station2}-{station1}-{number}')
        

            # Events
            
            # Arrival events forward (aef)
            events_pi_f[f'arr-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'arr-{station1}-{station2}-{number}')
            # Departure events forward (def)
            events_pi_f[f'dep-{station1}-{station2}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                             name=f'dep-{station1}-{station2}-{number}')
            # Arrival events forward (aeb)
            events_pi_b[f'arr-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'arr-{station2}-{station1}-{number}')
            
            # Departure events backwards (deb)
            events_pi_b[f'dep-{station2}-{station1}-{number}'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                                            name=f'dep-{station2}-{station1}-{number}')
            
            
    # Four Transfer activities at Eindhoven
    # Transfer activities backwards (tab)    
    transfer_activities[f'trans-Hrl-Ut-3500'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'trans-Hrl-Ut-3500')
    transfer_activities[f'trans-Hrl-Ut-800'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'trans-Hrl-Ut-800')
    
    # Transfer activities forward (taf)    
    transfer_activities[f'trans-Ut-Hrl-800'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'trans-Ut-Hrl-800')
    transfer_activities[f'trans-Ut-Hrl-3500'] = model.addVar(vtype=GRB.CONTINUOUS,
                                                        name=f'trans-Ut-Hrl-3500')
    
    headway_pairs = [
    ('800', '3500'),
    ('800', '3100'),
    ('3000', '3500'),
    ('3000', '3100')
    ]
    
    #Headway activities arrival or departure
    activity_types = ['a', 'd']
    
    # ha (a/d)
    for pair in headway_pairs:
        for activity_type in activity_types:
            key = f'ha{activity_type}-{pair[0]}-{pair[1]}'
            headway_activities[key] = model.addVar(vtype=GRB.CONTINUOUS, name=key)
    
    
    # Sync activities forwards/backwards (saf/sab)
    sync_pairs = [
    ('Amr_Asd_800', 'Amr_Asd_3000'),
    ('Asd_Ut_800', 'Asd_Ut_3000'),
    ('Ut_Ehv_800', 'Ut_Ehv_3500'),
    ('Shl_Ut_3100', 'Shl_Ut_3500'),
    ('Ut_Nm_3000', 'Ut_Nm_3100'),
    ('Ehv_Std_800', 'Ehv_Std_3900')
    ]

    # Define the types of headway activities
    direction = ['f', 'b']
    
    # Iterate over each pair and activity type to create variables
    for pair in sync_pairs:
        for d in direction:
            key = f'sa{d}-{pair[0]}-{pair[1]}'
            sync_activities[key] = model.addVar(vtype=GRB.CONTINUOUS, name=key)


    # Objective function: Minimize the total weighted travel time
    model.setObjective(
    quicksum(running_activites_f.values()) + 
    quicksum(dwelling_activities_f.values()) + 
    quicksum(transfer_activities.values()) + 
    quicksum(headway_activities.values()) + 
    quicksum(sync_activities.values()) + 
    quicksum(running_activites_b.values())+ 
    quicksum(dwelling_activities_b.values()), GRB.MINIMIZE)
    
    T_pij = 10
    
    # Constrains
    
    # running time constrains
    for k, activity_var in running_activites_f.items():
        parts = k.split('-')
        station1, station2 = parts[1], parts[2]
        result = travel_times.loc[(travel_times['From'] == station1) & (travel_times['To'] == station2), 'Travel Time'].values[0]
        
        model.addConstr(activity_var == result , name=f'{k}_constrain')
    
        # running time constrains
    for k,activity_var in running_activites_b.items():
        parts = k.split('-')
        station1, station2 = parts[1], parts[2]
        result = travel_times.loc[(travel_times['From'] == station2) & (travel_times['To'] == station1), 'Travel Time'].values[0]
        model.addConstr(activity_var == result, name=f'{k}_constrain')
        
        
       
    for k, activity_var in running_activites_f.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
        
        # Find the corresponding events for this activity
        start_event = events_pi_f[f'dep-{station1}-{station2}-{number}']
        end_event = events_pi_f[f'arr-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(activity_var == end_event - start_event + T_pij, name=f"duration_{k}")
    

    for k, activity_var in running_activites_b.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
        
        # Find the corresponding events for this activity
        start_event = events_pi_b[f'dep-{station1}-{station2}-{number}']
        end_event = events_pi_b[f'arr-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(activity_var == end_event - start_event + T_pij, name=f"duration_{k}")
        
        
        
        # dwelling time constrains 2 < activity < 8 (Bounds)
    for key,value in dwelling_activities_f.items():
        model.addConstr(2 <= dwelling_activities_f[key],
                name=f'{key}_constrain_2')
        model.addConstr(dwelling_activities_f[key] <= 8,
                name=f'{key}_constrain_8')
    
        # dwelling time constrains 2 < activity < 8 (Bounds)
    for key,value in dwelling_activities_b.items():
        model.addConstr(2 <= dwelling_activities_b[key],
                name=f'{key}_constrain_2')
        model.addConstr(dwelling_activities_b[key] <= 8,
                name=f'{key}_constrain_8')
    
    
    for k, d_var in dwelling_activities_f.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
    
        # Find the corresponding events for this activity
        start_event = events_pi_f[f'arr-{station1}-{station2}-{number}']
        end_event = events_pi_f[f'dep-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(d_var == end_event - start_event + T_pij, name=f"{k}")
        
    for k, d_var in dwelling_activities_b.items():
        # Parse activity keys to get the station names and number
        parts = k.split('-')
        station1, station2, number = parts[1], parts[2], parts[3]
    
        # Find the corresponding events for this activity
        start_event = events_pi_b[f'arr-{station1}-{station2}-{number}']
        end_event = events_pi_b[f'dep-{station1}-{station2}-{number}']
        
        # Create the constraint for this activity
        model.addConstr(d_var == end_event - start_event + T_pij, name=f"{k}")
        
        
    









    model.update()
    # print("Constraints:")
    # for constr in model.getConstrs():
    #     print(f"{constr.ConstrName}: {constr.sense} {constr.RHS}")

    
    return model


In [3]:
def solve_model(model):

    # Optimize the model
    model.optimize()

    #Store solutions in a dictionary
    solution_dict = {}

    #Collecting solutions for decision variables
    for var in model.getVars():
        solution_dict[var.varName] = var.x

    #Store Obj.function cost
    cost = model.objVal

    #Close the Gurobi model
    model.close()

    return solution_dict, cost

In [4]:
travel_times, lines, weights = read_basic_data()
model = build_model(travel_times, lines, weights)
solution_dict, cost = solve_model(model)


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-16
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11400H @ 2.70GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 130 rows, 142 columns and 238 nonzeros
Model fingerprint: 0xfcb9627c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 5e+01]
Presolve removed 86 rows and 98 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible model


AttributeError: Unable to retrieve attribute 'x'